In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pprint
import lxml
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

In [43]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    removed_rt = re.sub('RT ','',letters_only)
    lower_case = removed_rt.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [44]:
df = pd.read_csv('../dataset/tweets - fever - fear.csv')
df.head()

,text,weekday,month,day,hour,has_hashtag,has_url,fav_count,is_reply,retweet_count,followers,following,user,screen_name
0,RT @nadhyusoff99: @DabemePOP FEVER 🎊 #GFRIEND ...,Thu,Sep,12,15,1,0,0,0,3,1,7,genesis andrade,genesis91272665
1,🤦🏻‍♂️Waking up with a #102 #fever 🤦🏻‍♂️🤮🤬🤬🤬 to...,Thu,Sep,12,14,1,1,0,0,0,6,28,josephvybe303,josephvybe303
2,RT @buddies_rminder: @GFRDofficial Buddies are...,Thu,Sep,12,14,1,0,0,0,3,54,201,GFriendTime!! #FeverSeason🌵🌵,MultiNoo
3,RT @buddies_rminder: @GFRDofficial Buddies are...,Thu,Sep,12,14,1,0,0,0,3,644,211,Gin 銀河 | Fallin'Light,amanogawa_ginga
4,@GFRDofficial Buddies are you streaming #FEVER...,Thu,Sep,12,14,1,1,9,1,3,2758,110,Buddies REMINDER 05:00 AM KST,buddies_rminder


In [45]:
tweets = df['text']
tweets.head()

0    RT @nadhyusoff99: @DabemePOP FEVER 🎊 #GFRIEND ...
1    🤦🏻‍♂️Waking up with a #102 #fever 🤦🏻‍♂️🤮🤬🤬🤬 to...
2    RT @buddies_rminder: @GFRDofficial Buddies are...
3    RT @buddies_rminder: @GFRDofficial Buddies are...
4    @GFRDofficial Buddies are you streaming #FEVER...
Name: text, dtype: object

In [46]:
cleaned_tweets = []
for tweet in tweets:
    cleaned_tweets.append(tweet_cleaner(tweet))
cleaned_tweets[:5]

['fever gfriend sourcemusic buddy feverseason fever fallingli',
 'waking up with a fever to the one that got me sick aka lurubeca and julio',
 'rminder buddies are you streaming fever here is the current gfriend mv views for thek',
 'rminder buddies are you streaming fever here is the current gfriend mv views for thek',
 'buddies are you streaming fever here is the current gfriend mv views for thek']